In [108]:
import pandas as pd

In [109]:
file = "results_parsed_3.csv"

In [110]:
df = pd.read_csv(file)

In [111]:
df.head()

,Unnamed: 0,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Full_Address,location,point,lat,long
0,0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,NaN,"1803 Westview Ave, Danville Illinois","1803, Westview Avenue, Danville, Vermilion Cou...","(40.1555301092308, -87.61471508589744, 0.0)",40.155530,-87.614715
1,1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,NaN,"50 Henrietta Ave, Buffalo New York","50, Henrietta Avenue, Buffalo, Erie County, Ne...","(42.9563206122449, -78.89934789795919, 0.0)",42.956321,-78.899348
2,2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,NaN,"800 Allacante Dr, Ceres California","800, Allacante Drive, Ceres, Stanislaus County...","(37.590897, -120.981631, 0.0)",37.590897,-120.981631
3,3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,NaN,"1900 Vernon Ave, Modesto California","1900, Vernon Avenue, West Modesto, Modesto, St...","(37.614474926174495, -121.02792171140939, 0.0)",37.614475,-121.027922
4,4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,NaN,"500 Smart St, Houston Texas","Smart Living at Cypress Creek, Houston, Harris...","(29.95991275, -95.56226793134292, 0.0)",29.959913,-95.562268


In [112]:
df = df.drop(['Operations', 'Unnamed: 0', 'location', 'point'], axis=1)


df = df.rename(columns={"Incident ID": "incident_id"
                   , "Incident Date": "incident_date"
                   , "State": "state"
                   , "City Or County": "city_county"
                   , "Address": "street_address"
                   , "# Killed": "killed"
                   , "# Injured": "injured"
                  , "Full_Address": "full_address"})



In [113]:
cleaned_df = df.dropna(how='any')

In [99]:
#cleaned_df = dropped_df.set_index("incident_id")

In [114]:
cleaned_df

,incident_id,incident_date,state,city_county,street_address,killed,injured,full_address,lat,long
0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,"1803 Westview Ave, Danville Illinois",40.155530,-87.614715
1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,"50 Henrietta Ave, Buffalo New York",42.956321,-78.899348
2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,"800 Allacante Dr, Ceres California",37.590897,-120.981631
3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,"1900 Vernon Ave, Modesto California",37.614475,-121.027922
4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,"500 Smart St, Houston Texas",29.959913,-95.562268
...,...,...,...,...,...,...,...,...,...,...
1546,1893744,"January 3, 2021",Florida,Miami,10525 NW 24th Ave,0,8,"10525 NW 24th Ave, Miami Florida",25.871241,-80.238231
1547,1892114,"January 1, 2021",Iowa,Sioux City,2637 S Walker St,1,4,"2637 S Walker St, Sioux City Iowa",42.462977,-96.331669
1548,1892034,"January 1, 2021",Arkansas,Fort Smith,3211 Towson Ave,0,7,"3211 Towson Ave, Fort Smith Arkansas",35.353577,-94.422035
1549,1892704,"January 1, 2021",Texas,Amarillo,2650 Dumas Dr,1,3,"2650 Dumas Dr, Amarillo Texas",35.240877,-101.830876


In [43]:
# dropped_df.to_csv("cleaned.csv")

In [104]:
from sqlalchemy import create_engine

rds_connection_string = "postgres:postgres@localhost:5432/mass_shootings_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [105]:
engine.table_names()

<ipython-input-105-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Mass_Shootings']

In [106]:
cleaned_df.to_sql(name='Mass_Shootings', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "incident_id" of relation "Mass_Shootings" violates not-null constraint
DETAIL:  Failing row contains (null, December 29, 2019, Illinois, Danville, 1803  Westview Ave, 0, 5, 1803  Westview Ave, Danville Illinois, 40.1555301092308, -87.61471508589744).

[SQL: INSERT INTO "Mass_Shootings" (incident_date, state, city_county, street_address, killed, injured, full_address, lat, long) VALUES (%(incident_date)s, %(state)s, %(city_county)s, %(street_address)s, %(killed)s, %(injured)s, %(full_address)s, %(lat)s, %(long)s)]
[parameters: ({'incident_date': 'December 29, 2019', 'state': 'Illinois', 'city_county': 'Danville', 'street_address': '1803  Westview Ave', 'killed': 0, 'injured': 5, 'full_address': '1803  Westview Ave, Danville Illinois', 'lat': 40.1555301092308, 'long': -87.61471508589744}, {'incident_date': 'December 29, 2019', 'state': 'New York', 'city_county': 'Buffalo', 'street_address': '50  Henrietta Ave', 'kil